In [11]:
import pybullet as p
import time
import pybullet_data
import os
import numpy as np
import pandas as pd
import math
import random
import sympy as sym
from sympy.parsing.sympy_parser import parse_expr
import pickle

In [ ]:
#Especificaciones brazo ur5e

#6 DOF (6 revolute joints, 6 joint variables)
#Q=(theta_1, theta_2, theta_3, theta_4, theta_5, theta_6)T

dicc = {'link':[i for i in range(1, 7)], 'a_i':[0, 0.425, 0.39225, 0, 0, 0],
        'alpha_i':[math.pi/2, 0, 0, math.pi/2, -math.pi/2, 0],
        'd_i':[0.089159, 0, 0, 0.10915, 0.09465, 0.0823], 
        'theta_i':[sym.Symbol('theta_1'), sym.Symbol('theta_2'), sym.Symbol('theta_3'), 
                   sym.Symbol('theta_4'), sym.Symbol('theta_5'), sym.Symbol('theta_6')],
        'd_theta_i':[sym.Symbol('d_theta_1'), sym.Symbol('d_theta_2'), sym.Symbol('d_theta_3'), 
                   sym.Symbol('d_theta_4'), sym.Symbol('d_theta_5'), sym.Symbol('d_theta_6')],
        'dd_theta_i':[sym.Symbol('dd_theta_1'), sym.Symbol('dd_theta_2'), sym.Symbol('dd_theta_3'), 
                sym.Symbol('dd_theta_4'), sym.Symbol('dd_theta_5'), sym.Symbol('dd_theta_6')]}

df_manipulator_parameters=pd.DataFrame(dicc)
df_manipulator_parameters.set_index(keys="link", inplace=True, drop=True)
display(df_manipulator_parameters)

,a_i,alpha_i,d_i,theta_i,d_theta_i,dd_theta_i
link,,,,,,
1,0.00000,1.570796,0.089159,theta_1,d_theta_1,dd_theta_1
2,0.42500,0.000000,0.000000,theta_2,d_theta_2,dd_theta_2
3,0.39225,0.000000,0.000000,theta_3,d_theta_3,dd_theta_3
4,0.00000,1.570796,0.109150,theta_4,d_theta_4,dd_theta_4
5,0.00000,-1.570796,0.094650,theta_5,d_theta_5,dd_theta_5
6,0.00000,0.000000,0.082300,theta_6,d_theta_6,dd_theta_6


In [ ]:
#Matriz de rotación
def R(angle, axis):
    s=sym.sin(angle)
    c=sym.cos(angle)

    if(axis=='x'):
        R_3x3=sym.Matrix([[1,0,0], [0, c, -s], [0, s, c]])
    elif(axis=='y'):
        R_3x3=sym.Matrix([[c,0,s], [0, 1, 0], [-s, 0, c]])
    elif(axis=='z'):
        R_3x3=sym.Matrix([[c,-s,0], [s, c, 0], [0, 0, 1]])
    
    R = R_3x3.row_join(sym.Matrix([[0], [0], [0]]))
    R = R.col_join(sym.Matrix([[0, 0, 0, 1]]))
    
    return(R)

In [ ]:
#Matriz de traslación
def D(d, axis_index):
    D=sym.eye(4)
    D[-1, -1]=1
    D[axis_index, -1]=d

    return(D)
    

In [ ]:
#Transformada homogénea
def homogeneous_transform(theta_i, alpha_i, a_i, d_i):
    return sym.simplify(D(d_i, 2)@R(theta_i, 'z')@D(a_i, 0)@R(alpha_i, 'x'))


In [ ]:
#Propagación de velocidades
def velocity_prop(j_v_j, j_w_j, j_plus_1_R_j, j_P_j_plus_1, theta_j_plus_1): #min j is 0
    j_Z_j = sym.Matrix([[0], [0], [1]])
    d_theta_j_plus_1=sym.Symbol('d_'+str(theta_j_plus_1))
    
    j_plus_1_w_j_plus_1=sym.simplify(j_plus_1_R_j@(j_w_j+d_theta_j_plus_1*j_Z_j))
    j_plus_1_v_j_plus_1=sym.simplify(j_plus_1_R_j@(j_v_j+sym.Matrix(np.cross((j_w_j+d_theta_j_plus_1*j_Z_j).reshape(1,3), j_P_j_plus_1.reshape(1,3)).reshape(3,1))))
    
    
    return(j_plus_1_w_j_plus_1, j_plus_1_v_j_plus_1)

In [ ]:
#Cinemática directa y velocidades de los enlaces
def direct_kinematics_and_link_velocities():

    v_link_vector=np.zeros((3,1)) #0v0 #Vector de velocidades linelaes de los enlaces
    w_link_vector=np.zeros((3,1)) #0w0 #Vector de velocidades angulares de los enlaces
    
    for i in range(1,7): #min i is 1
        theta_i=df_manipulator_parameters.loc[i, 'theta_i']
        alpha_i=df_manipulator_parameters.loc[i, 'alpha_i']
        a_i=df_manipulator_parameters.loc[i, 'a_i']
        d_i=df_manipulator_parameters.loc[i, 'd_i']
        
        i_minus_1_T_i=homogeneous_transform(theta_i, alpha_i, a_i, d_i)
        
        dicc_transformadas[str(i-1)+'T'+str(i)]=i_minus_1_T_i

        #Para calcular las velocidades de los enlaces, j=i-1
        j_plus_1_R_j = i_minus_1_T_i[0:3, 0:3].reshape(3,3)@sym.Matrix([[1, 0, 0], [0, -1, 0], [0, 0, 1]]) #Inversa de i_minus_1_T_i,
        #Para la primera iteración, se obtiene 1_R_0 de 0_T_1
        j_P_j_plus_1 = i_minus_1_T_i[0:3, 3].reshape(3,1) #Vector de posición
        
        #propagación de velocidades
        w, v=velocity_prop(j_v_j=v_link_vector[0:3,i-1].reshape(3,1), j_w_j=w_link_vector[0:3,i-1].reshape(3,1), 
                           j_plus_1_R_j=j_plus_1_R_j, j_P_j_plus_1=j_P_j_plus_1, theta_j_plus_1=theta_i)
        
        #print('\nv: ', v)
        #print('\nw: ', w)
        
        v_link_vector=np.append(v_link_vector, v, axis=1)
        w_link_vector=np.append(w_link_vector, w, axis=1)
        #print('vector velocidades: ', v_link_vector)
    
    T_total=sym.eye(4) #Inicialización trasformada total
    
    for transformada in (dicc_transformadas.values()):
        T_total=transformada@T_total
 
    T_total=sym.simplify(T_total) #Simplifica expresión trasnformada total
    
    #Cáculo de las velocidades del efector final desde el sistema {0}
    o_R_ef = T_total[0:3, 0:3].reshape(3,3) #T_total es 0_T_ef
    
    o_v_ef = o_R_ef@v_link_vector[0:3, -1].reshape(3,1) #v_link_vector[0:2, -1] es la velocidad lineal del último eslabón, ef_v_ef
    o_w_ef = o_R_ef@w_link_vector[0:3, -1].reshape(3,1) #w_link_vector[0:2, -1] es la velocidad angular del último eslabón, ef_w_ef  
    
    return(T_total, v_link_vector, w_link_vector, o_v_ef, o_w_ef)

In [ ]:
#Modelo dinámico
def dynamic_model(v_link_vector, w_link_vector, CoM_ref_base, masas, M_total, B, Fc, r):
    
    ##########################################################################
    
    #Cálculo matriz de Coriolis
    dicc_derivadas_Coriolis={}
    for i in range(0, 6):
        for j in range(0, 6):
            dicc_derivadas_Coriolis[str(i)+str(j)]=[sym.diff(M_total[i][j], theta_i) for theta_i in df_manipulator_parameters['theta_i']]
            
    #print(dicc_derivadas_Coriolis)        
            
    
    for i in range(0, 6):
        for j in range(0, 6):
            for k in range(0,6):       
                Coriolis[i,j]=Coriolis[i,j]+1/2*(dicc_derivadas_Coriolis[str(i)+str(j)][k]+dicc_derivadas_Coriolis[str(i)+str(k)][j]-dicc_derivadas_Coriolis[str(j)+str(k)][i])*sym.Symbol('theta_'+str(k+1))
    
    #print('\nHa calculado Coriolis')

    ##########################################################################
    
    #Cálculo energías cinética y potencial del sistema
    
    Ec_T=0
    Ep_T=0
    for i in range(1, 6): #i=0 es la base
        v=v_link_vector[0:3, i]
        w=w_link_vector[0:3, i]
        I=TI_total[i] 
        
        #Energía cinética (traslación y rotación)
        Ec_T=1/2*(Ec_T+masas[i]*v@np.transpose(v)+np.transpose(w)@I@w)
        
        #Energía potencial
        Ep_T=Ep_T+masas[i]*(CoM_ref_base[i,:]@sym.Matrix([[0], [0], [-9.81]]))[0]

    #print('\nHa calculado energías')   

    ##########################################################################   
  
    #Vector G
    G = sym.Matrix([sym.diff(Ep_T, theta_i) for theta_i in df_manipulator_parameters['theta_i']])
     
    #print('\nHa calculado G')
    
   ##########################################################################
    
    #Ecuaciones dinámicas
    
    tau=M_total@dd_theta_array.reshape(6,1)+Coriolis@d_theta_array.reshape(6,1)+G.reshape(6,1)

    print('\nHa calculado tau')
    
    return(tau)

In [ ]:

#Función para expandir las dimensiones 
#de las matrices de inercia de (3x3) a (6x6)

def expand_inertia_matrix(TI, com, m): 
    
    #Matriz cruzada del centro de masa
    com_cross = np.array([
    [0, -com[2], com[1]],
    [com[2], 0, -com[0]],
    [-com[1], com[0], 0]])
    
    #Producto de la masa con la matriz cruzada                    
    m_com_cross=m*com_cross
    
    #Cálculo de las submatrices
    upper_left = TI + m_com_cross @ np.transpose(com_cross)
    upper_right = m_com_cross
    lower_left = np.transpose(m_com_cross)
    lower_right = m * np.eye(3)
    
    #Se construye el tensor expandido
    upper_block = np.hstack((upper_left, upper_right))
    lower_block = np.hstack((lower_left, lower_right))
    TI_expanded = np.vstack((upper_block, lower_block))
    
    return(TI_expanded)

## MAIN

In [ ]:
dicc_transformadas={}

#Cinemática directa
T_total, v_link_vector, w_link_vector, o_v_ef, o_w_ef=direct_kinematics_and_link_velocities()

In [ ]:
#Inicialización parámetros

theta_1=sym.Symbol('theta_1')
theta_2=sym.Symbol('theta_2')
theta_3=sym.Symbol('theta_3')
theta_4=sym.Symbol('theta_4')
theta_5=sym.Symbol('theta_5')
theta_6=sym.Symbol('theta_6')

d_theta_1=sym.Symbol('d_theta_1')
d_theta_2=sym.Symbol('d_theta_2')
d_theta_3=sym.Symbol('d_theta_3')
d_theta_4=sym.Symbol('d_theta_4')
d_theta_5=sym.Symbol('d_theta_5')
d_theta_6=sym.Symbol('d_theta_6')

dd_theta_1=sym.Symbol('dd_theta_1')
dd_theta_2=sym.Symbol('dd_theta_2')
dd_theta_3=sym.Symbol('dd_theta_3')
dd_theta_4=sym.Symbol('dd_theta_4')
dd_theta_5=sym.Symbol('dd_theta_5')
dd_theta_6=sym.Symbol('dd_theta_6')


dd_theta_array=sym.Matrix([dd_theta_1, dd_theta_2, dd_theta_3, 
            dd_theta_4, dd_theta_5, dd_theta_6])
    
d_theta_array=sym.Matrix([d_theta_1, d_theta_2, d_theta_3, 
            d_theta_4, d_theta_5, d_theta_6])


#Centros de masa, referenciados a los sistemas locales

CoM=np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.2125], [0.0, 0.0, 0.196], 
              [0.0, 0.127, 0.0], [0.0, 0.0, 0.1], [0.0, 0.0771, 0.0], [0.0, 0.0, 0.0]])


#Masas enlaces

masas=np.array([4.0, 3.7, 8.393, 2.275, 1.219, 1.219, 0.1879, 0.636951])

r=0 #coeficiente de reducción para SVD

#Tensores de inercia
TI_0=np.array([[0.00443333156, 0.0, 0.0], [0.0, 0.00443333156, 0.0], [0.0, 0.0, 0.0072]])
TI_1=np.array([[0.010267495893, 0.0, 0.0], [0.0, 0.010267495893, 0.0], [0.0, 0.0, 0.00666]])
TI_2=np.array([[0.133885781862, 0.0, 0.0], [0.0, 0.133885781862, 0.0], [0.0, 0.0, 0.0151074]])
TI_3=np.array([[0.0311796208615, 0.0, 0.0], [0.0, 0.0311796208615, 0.0], [0.0, 0.0, 0.004095]])
TI_4=np.array([[0.00255989897604, 0.0, 0.0], [0.0, 0.00255989897604, 0.0], [0.0, 0.0, 0.0021942]])
TI_5=np.array([[0.00255989897604, 0.0, 0.0], [0.0, 0.00255989897604, 0.0], [0.0, 0.0, 0.0021942]])
TI_6=np.array([[9.89041005217e-05, 0.0, 0.0], [0.0, 9.89041005217e-05, 0.0], [0.0, 0.0, 0.0001321171875]])
TI_ef=np.array([[0.000380, 0.0, 0.0], [0.0, 0.001110, 0.0], [0.0, 0.0, 0.001171]])

#Matrices rotación respecto al sistema de la base
R01 = dicc_transformadas['0T1'][:3,:3]
R02 = dicc_transformadas['1T2'][:3,:3] @ R01  
R03 = dicc_transformadas['2T3'][:3,:3] @ R02  
R04 = dicc_transformadas['3T4'][:3,:3] @ R03 
R05 = dicc_transformadas['4T5'][:3,:3] @ R04
R06 = dicc_transformadas['5T6'][:3,:3] @ R05  

#Tensored de inercia respecto al sistema de la base
TI_1_base = R01 @ TI_1 @ np.transpose(R01)
TI_2_base = R02 @ TI_2 @ np.transpose(R02)
TI_3_base = R03 @ TI_3 @ np.transpose(R03)
TI_4_base = R04 @ TI_4 @ np.transpose(R04)
TI_5_base = R05 @ TI_5 @ np.transpose(R05)
TI_6_base = R06 @ TI_6 @ np.transpose(R06)

#Vector tensores
TI_total=np.array([TI_0, TI_1_base, TI_2_base, TI_3_base, TI_4_base, TI_5_base, TI_6_base])

#Referenciar los centros de masa con respecto al sistema de la base
CoM_ref_base=sym.Matrix([[0.0, 0.0, 0.0]]) #Inicializado con el CoM de la base
T_base=sym.eye(4)

for i in range(1,7):
    T_base=dicc_transformadas[str(i-1)+'T'+str(i)]@T_base
    CoM_ref_base=CoM_ref_base.row_insert(i, (sym.Matrix(np.append(CoM[i+1], np.array([1.0]), axis=0).reshape(1,4))@T_base)[:, 0:3])

#Expansión tensores de inercia
TI_1_base_expand=expand_inertia_matrix(TI_1_base, CoM_ref_base[1, 0:3], masas[1]) #El com y la masa de la posición 0 corresponde a la base del manipulador
TI_2_base_expand=expand_inertia_matrix(TI_1_base, CoM_ref_base[2, 0:3], masas[2])
TI_3_base_expand=expand_inertia_matrix(TI_1_base, CoM_ref_base[3, 0:3], masas[3])
TI_4_base_expand=expand_inertia_matrix(TI_1_base, CoM_ref_base[4, 0:3], masas[4])
TI_5_base_expand=expand_inertia_matrix(TI_1_base, CoM_ref_base[5, 0:3], masas[5])
TI_6_base_expand=expand_inertia_matrix(TI_1_base, CoM_ref_base[6, 0:3], masas[6])

# Suma de las matrices de inercia para obtener la matriz de inercia total
M_total = TI_1_base_expand + TI_2_base_expand + TI_3_base_expand + TI_4_base_expand + TI_5_base_expand + TI_6_base_expand

# Simplificar la matriz de inercia total
M_total = sym.simplify(M_total)
M_total=sym.Matrix(M_total)

#Inicialización matriz Coriolis
Coriolis=sym.zeros(6, 6)

#Inicialización vector G
G=sym.zeros(6, 1)

#Supongamos que la fricción viscosa y de Coulomb de las articulaciones son 0
B=np.zeros((6,1)) #Fricción viscosa
Fc=np.zeros((6,1)) #Coulomb


In [ ]:
tau=dynamic_model(v_link_vector, w_link_vector, CoM_ref_base, masas, M_total, B, Fc, r)